# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

Mounted at /content/drive


In [ ]:
import re
from time import sleep
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from packaging import version
from IPython import display
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras import layers, losses, Model, Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
sns.set(style="whitegrid")

# Diretórios base
BASE_DIR = os.path.dirname(os.path.abspath(''))
DATASET_DIR = os.path.join(BASE_DIR, 'datasets')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
HISTORY_DIR = os.path.join(BASE_DIR, 'history')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

# Dicionário de grupos funcionais
func_grp_smarts = {
    'alkane':'[CX4;H0,H1,H2,H4]',
    'methyl':'[CH3]',
    'alkene':'[CX3]=[CX3]',
    'alkyne':'[CX2]#C',
    'alcohols':'[#6][OX2H]',
    'amines':'[NX3;H2,H1;!$(NC=O)]',
    'nitriles':'[NX1]#[CX2]',
    'aromatics':'[$([cX3](:*):*),$([cX2+](:*):*)]',
    'alkyl halides':'[#6][F,Cl,Br,I]',
    'esters':'[#6][CX3](=O)[OX2H0][#6]',
    'ketones':'[#6][CX3](=O)[#6]',
    'aldehydes':'[CX3H1](=O)[#6]',
    'carboxylic acids':'[CX3](=O)[OX2H1]',
    'ether': '[OD2]([#6])[#6]',
    'acyl halides':'[CX3](=[OX1])[F,Cl,Br,I]',
    'amides':'[NX3][CX3](=[OX1])[#6]',
    'nitro':'[$([NX3](=O)=O),$([NX3+](=O)[O-])][!#8]'
}

column_names = list(func_grp_smarts.keys())

# Caminhos dos arquivos
DATASET_SPECTRA_PARQUET = os.path.join(DATASET_DIR, 'df_spectra_all_mixture_interpolate.parquet')
DATASET_ENRICH_CSV = os.path.join(DATASET_DIR, 'df_enrich.csv')
AUTOENCODER_MODEL_PATH = os.path.join(MODELS_DIR, 'autoencoder_model_2025_04_07.keras')

In [ ]:
autoencoder_load = keras.models.load_model(AUTOENCODER_MODEL_PATH)
autoencoder_load.summary()
encoder_model_load = keras.Model(inputs=autoencoder_load.input, outputs=autoencoder_load.layers[4].output)  # Ajuste se necessário

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 1030)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 512)            │       527,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 25)             │         3,225 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 128)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 1030)           │       528,390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,174,943 (15.93 MB)

 Trainable params: 1,391,647 (5.31 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,783,296 (10.62 MB)

In [ ]:
df_enrich = pd.read_csv(DATASET_ENRICH_CSV)
dataset_y = df_enrich.copy()
dataset_y.index = dataset_y['CAS']
len(dataset_y.CAS.unique())

8241

In [ ]:
df_spectra_all = pd.read_parquet(DATASET_SPECTRA_PARQUET)
mean_cols = [x for x in df_spectra_all.columns if 'mean' in x]
min_cols = [x for x in df_spectra_all.columns if 'min' in x]
max_cols = [x for x in df_spectra_all.columns if 'max' in x]
all_cols = mean_cols + min_cols + max_cols
df_spectra_all.shape

(1030, 24636)

# Functions

In [6]:
def get_dataset(df_spectra_all, dataset_y, agg_func):

  if agg_func == 'mean':
    dataset_x = df_spectra_all[mean_cols].copy()
  elif agg_func == 'min':
    dataset_x = df_spectra_all[min_cols].copy()

  elif agg_func == 'max':
    dataset_x = df_spectra_all[max_cols].copy()

  # dataset_x = df_spectra_all.copy()
  dataset_x = dataset_x.T
  dataset_x.columns = ['bin_' + str(x) for x in dataset_x.columns]
  dataset_x.reset_index(inplace=True)
  dataset_x.index = dataset_x['index'].apply(lambda x: x.split('_')[0])

  dataset_y = dataset_y[dataset_y['yunits'] == 'ABSORBANCE']

  dataset_final = pd.merge(dataset_y, dataset_x, left_index = True, right_index = True, how='inner')

  return dataset_final

def find_best_epoch(history):
    """
    Finds the epoch with the lowest validation loss.

    Args:
        history: Training history object from Keras model.fit().

    Returns:
        A tuple containing the best epoch number and its corresponding validation loss.
        Returns None if history object is invalid or empty.
    """
    if not history or 'val_loss' not in history.history:
        return None

    val_losses = history.history['val_loss']
    best_epoch = np.argmin(val_losses)  # Index of the minimum validation loss
    best_val_loss = val_losses[best_epoch]

    return best_epoch, best_val_loss

In [7]:
current_date = datetime.now().strftime('%Y_%m_%d')

In [ ]:
def compute_model_analysis(agg_func, data_prep, current_date, batch_size=600, epochs=100, callbacks=False, save_history=True, save_plot=False):
    dataset_final = get_dataset(df_spectra_all, dataset_y, agg_func)
    X = dataset_final[[col for col in dataset_final.columns if 'bin' in col]]
    X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
    if data_prep == 'encoder':
        X = encoder_model_load.predict(X)
    Y = dataset_final[column_names].apply(lambda x: x.astype(int))
    X_train, X_test_temp, Y_train, Y_test_temp = train_test_split(X, Y, test_size=0.20, random_state=42, stratify=Y[column_names].sum(axis=1))
    X_validation, X_test, Y_validation, Y_test = train_test_split(X_test_temp, Y_test_temp, test_size=0.30, random_state=42, stratify=Y_test_temp[column_names].sum(axis=1))
    data_type = np.float32
    # Reshape input data for GRU
    X_train = np.array(X_train, dtype=data_type).reshape((X_train.shape[0], X_train.shape[1], 1))
    X_validation = np.array(X_validation, dtype=data_type).reshape((X_validation.shape[0], X_validation.shape[1], 1))
    X_test = np.array(X_test, dtype=data_type).reshape((X_test.shape[0], X_test.shape[1], 1))
    X_test_temp = np.array(X_test_temp, dtype=data_type).reshape((X_test_temp.shape[0], X_test_temp.shape[1], 1))
    # Define the GRU model
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1], 1)),
        layers.GRU(64, return_sequences=True),
        layers.Dropout(0.2),
        layers.GRU(32, return_sequences=True),
        layers.Dropout(0.2),
        layers.GRU(32),
        layers.Dense(16, activation='relu'),
        layers.Dense(Y_train.shape[1], activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'binary_accuracy', 'precision', 'recall', 'binary_crossentropy'])
    # Callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min', start_from_epoch=50, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-4, verbose=0, mode='min')
    callbacks_list = [early_stopping, reduce_lr] if callbacks else []
    history = model.fit(X_train, Y_train,
                      validation_data=(X_validation, Y_validation),
                      callbacks=callbacks_list,
                      epochs=epochs, verbose=1, shuffle=True, batch_size=batch_size)
    # Salvar modelo e histórico
    compose_name = f'gru_{agg_func}_{data_prep}_{current_date}_call_{callbacks}'
    model.save(os.path.join(MODELS_DIR, f'{compose_name}.keras'))
    if save_history:
        with open(os.path.join(HISTORY_DIR, f'{compose_name}_history.pkl'), 'wb') as f:
            pickle.dump(history.history, f)
    best_epoch, best_val_loss = find_best_epoch(history)
    if save_plot:
        plt.figure(figsize=(16, 10))
        plt.suptitle(f'Métricas Treinamento - Época Escolhida {best_epoch} - {best_val_loss:.3f} Validação Loss Function')
        plt.subplot(3, 1, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.axvline(x=best_epoch, color='red', linestyle='--')
        plt.ylabel('Acurácia')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: '{:.3f}'.format(x)))
        plt.subplot(3, 1, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.axvline(x=best_epoch, color='red', linestyle='--')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: '{:.3f}'.format(x)))
        plt.subplot(3, 1, 3)
        plt.plot(history.history['binary_accuracy'])
        plt.plot(history.history['val_binary_accuracy'])
        plt.axvline(x=best_epoch, color='red', linestyle='--')
        plt.ylabel('Acurácia Binarizada')
        plt.xlabel('Epoch')
        plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: '{:.3f}'.format(x)))
        plt.savefig(os.path.join(RESULTS_DIR, f'{compose_name}.png'))
        plt.show()
    return history

In [9]:
batch_size = 360
epochs = 100
callbacks = False
# compute_model_analysis('min', 'normal', current_date, batch_size = batch_size,epochs = epochs, callbacks=callbacks)
# compute_model_analysis('min', 'encoder', current_date, batch_size = batch_size,epochs = epochs, callbacks=callbacks)
# compute_model_analysis('max', 'normal', current_date, batch_size = batch_size,epochs = epochs, callbacks=callbacks)
# compute_model_analysis('max', 'encoder', current_date, batch_size = batch_size,epochs = epochs, callbacks=callbacks)
compute_model_analysis('mean', 'normal', current_date, batch_size = batch_size,epochs = epochs, callbacks=callbacks)
compute_model_analysis('mean', 'encoder', current_date, batch_size = batch_size,epochs = epochs, callbacks=callbacks)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 139s 7s/step - accuracy: 0.1034 - binary_accuracy: 0.6245 - binary_crossentropy: 0.6868 - f1_score: 0.0304 - loss: 0.6868 - precision: 0.3018 - recall: 0.6608 - val_accuracy: 0.0322 - val_binary_accuracy: 0.6015 - val_binary_crossentropy: 0.6403 - val_f1_score: 0.0254 - val_loss: 0.6403 - val_precision: 0.2878 - val_recall: 0.6813
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.5481 - binary_accuracy: 0.6172 - binary_crossentropy: 0.6213 - f1_score: 0.0568 - loss: 0.6213 - precision: 0.2945 - recall: 0.6636 - val_accuracy: 0.0113 - val_binary_accuracy: 0.6882 - val_binary_crossentropy: 0.5624 - val_f1_score: 0.0111 - val_loss: 0.5624 - val_precision: 0.3399 - val_recall: 0.6036
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.1939 - binary_accuracy: 0.6765 - binary_crossentropy: 0.5487 - f1_score: 0.0314 - loss: 0.5487 - precision: 0.3234 - recall: 0.5770 - val_accuracy: 0.7041 - val_binary_accuracy: 0.7715 - val

# Comparações

In [10]:
def compute_general_metrics(y_real,y_pred,i=None):

  if i is None:
    names = 'general'
    accur = metrics.accuracy_score(y_real, y_pred)
    f1 = metrics.f1_score(y_real, y_pred, average='weighted')
    prec = metrics.precision_score(y_real, y_pred, average='weighted')
    rec = metrics.recall_score(y_real, y_pred, average='weighted')
    jacc = float(metrics.jaccard_score(y_real, y_pred, average='weighted'))
    hamm = metrics.hamming_loss(y_real, y_pred)
    log_loss = metrics.log_loss(y_real, y_pred)

  elif i is not None:
    names = column_names[i]
    accur = metrics.accuracy_score(y_real.iloc[:, i], y_pred[:, i])
    f1 = metrics.f1_score(y_real.iloc[:, i], y_pred[:, i],zero_division=0)
    prec = metrics.precision_score(y_real.iloc[:, i], y_pred[:, i],zero_division=0)
    rec = metrics.recall_score(y_real.iloc[:, i], y_pred[:, i],zero_division=0)
    jacc = float(metrics.jaccard_score(y_real.iloc[:, i], y_pred[:, i]))
    hamm = metrics.hamming_loss(y_real.iloc[:, i], y_pred[:, i])
    log_loss = metrics.log_loss(y_real.iloc[:, i], y_pred[:, i])

  return [names, accur, f1, prec, rec, jacc, hamm, log_loss]

In [ ]:
def compute_all_comparisons(func_model, data_model):
    model_prefix = f'gru_{func_model}_{data_model}'
    model_suffix = f'_2025_06_28_call_True'  # Ajuste conforme necessário
    model_name = model_prefix + model_suffix
    model_path = os.path.join(MODELS_DIR, f'{model_name}.keras')
    model = keras.models.load_model(model_path)
    dataset_final = get_dataset(df_spectra_all, dataset_y, func_model)
    X = dataset_final[[col for col in dataset_final.columns if 'bin' in col]]
    X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
    if data_model == 'encoder':
        X = encoder_model_load.predict(X)
    Y = dataset_final[column_names].apply(lambda x: x.astype(int))
    X_train, X_test_temp, Y_train, Y_test_temp = train_test_split(X, Y, test_size=0.25, random_state=42)
    X_validation, X_test, Y_validation, Y_test = train_test_split(X_test_temp, Y_test_temp, test_size=0.45, random_state=42)
    data_type = np.float32
    X_train = np.array(X_train, dtype=data_type).reshape((X_train.shape[0], X_train.shape[1], 1))
    X_validation = np.array(X_validation, dtype=data_type).reshape((X_validation.shape[0], X_validation.shape[1], 1))
    X_test = np.array(X_test, dtype=data_type).reshape((X_test.shape[0], X_test.shape[1], 1))
    X_test_temp = np.array(X_test_temp, dtype=data_type).reshape((X_test_temp.shape[0], X_test_temp.shape[1], 1))
    train_predict = (model.predict(X_train) > 0.5).astype(int)
    test_predict = (model.predict(X_test_temp) > 0.5).astype(int)
    metrics_list = ['accuracy', 'f1_score', 'precision', 'recall', 'jaccard', 'hamming', 'log_loss']
    train_metrics = [compute_general_metrics(Y_train, train_predict)] + [compute_general_metrics(Y_train, train_predict, x) for x in range(len(column_names))]
    test_metrics = [compute_general_metrics(Y_test_temp, test_predict)] + [compute_general_metrics(Y_test_temp, test_predict, x) for x in range(len(column_names))]
    full_metrics_df = pd.DataFrame(train_metrics)
    full_metrics_df.columns = ['metric'] + metrics_list
    full_metrics_df['data'] = 'train'
    test_metrics_df = pd.DataFrame(test_metrics)
    test_metrics_df.columns = ['metric'] + metrics_list
    test_metrics_df['data'] = 'test'
    merged_df = pd.concat([full_metrics_df, test_metrics_df], ignore_index=True)
    merged_df['model'] = model_prefix
    return merged_df

results_list = []
for func_model in ['min', 'max', 'mean']:
    for data_model in ['normal', 'encoder']:
        results_list.append(compute_all_comparisons(func_model, data_model))

193/193 ━━━━━━━━━━━━━━━━━━━━ 51s 263ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 18s 278ms/step


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


193/193 ━━━━━━━━━━━━━━━━━━━━ 53s 273ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 17s 267ms/step


...

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


193/193 ━━━━━━━━━━━━━━━━━━━━ 52s 265ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 17s 259ms/step


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [12]:
all_metrics = pd.concat(results_list) #.groupby(['model','data','metric']).mean()

In [13]:
all_metrics.to_csv('all_metrics_gru.csv',index=False)

In [14]:
all_metrics = pd.concat(results_list) #.groupby(['model','data','metric']).mean()
all_metrics[(all_metrics.model.isin(['gru_min_encoder','gru_min_normal'])) & (all_metrics['data'].isin(['test']))]

,metric,accuracy,f1_score,precision,recall,jaccard,hamming,log_loss,data,model
18,general,0.030687,0.444695,0.365836,0.569073,0.365836,0.149107,52.301044,test,gru_min_normal
19,alkane,0.698003,0.822146,0.698003,1.000000,0.698003,0.301997,10.885078,test,gru_min_normal
20,methyl,0.645397,0.784488,0.645397,1.000000,0.645397,0.354603,12.781188,test,gru_min_normal
21,alkene,0.878714,0.000000,0.000000,0.000000,0.000000,0.121286,4.371588,test,gru_min_normal
22,alkyne,0.971749,0.000000,0.000000,0.000000,0.000000,0.028251,1.018281,test,gru_min_normal
23,alcohols,0.728690,0.000000,0.000000,0.000000,0.000000,0.271310,9.779014,test,gru_min_normal
24,amines,0.895762,0.000000,0.000000,0.000000,0.000000,0.104238,3.757108,test,gru_min_normal
25,nitriles,0.954213,0.000000,0.000000,0.000000,0.000000,0.045787,1.650318,test,gru_min_normal
26,aromatics,0.572820,0.728399,0.572820,1.000000,0.572820,0.427180,15.397118,test,gru_min_normal
27,alkyl halides,0.731125,0.000000,0.000000,0.000000,0.000000,0.268875,9.691231,test,gru_min_normal


In [ ]:
all_metrics.to_csv('all_metrics_gru.csv',index=False)